<a href="https://colab.research.google.com/github/arva29/AMDproject/blob/main/UkraineWarTweetsProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# this is meant to be run on google colab

!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#!wget -q http://www-eu.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!wget -q https://archive.apache.org/dist/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz
!tar xf spark-2.4.5-bin-hadoop2.7.tgz
!pip install -q findspark

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

import findspark
findspark.init("spark-2.4.5-bin-hadoop2.7") #SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

sc = spark.sparkContext

In [3]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 70 bytes


In [4]:
!kaggle datasets download -d bwandowando/ukraine-russian-crisis-twitter-dataset-1-2-m-rows

100% 12.4G/12.4G [04:07<00:00, 83.4MB/s]
100% 12.4G/12.4G [04:07<00:00, 53.6MB/s]


In [5]:
!unzip ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip

Archive:  ukraine-russian-crisis-twitter-dataset-1-2-m-rows.zip
  inflating: 0819_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0820_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0821_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0822_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0823_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0824_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0825_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0826_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0827_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0828_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0829_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0830_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0831_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0901_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0902_UkraineCombinedTweetsDeduped.csv.gzip  
  inflating: 0903_UkraineCombinedTweetsDeduped.csv.gzip  
  inflat

I extract the collection of tweets from the (23 of September ?? ) and I will work on this.

In [6]:
import csv
import gzip

data_list = []

with gzip.open('1009_UkraineCombinedTweetsDeduped.csv.gzip', 'rt') as f:
    reader = csv.DictReader(f)
    for row in reader:
        data_list.append(row)

rdd = sc.parallelize(data_list)

In [7]:
rdd.count()

66366

In [8]:
rdd.first()

OrderedDict([('', '101923'),
             ('userid', '1573729011364241409'),
             ('username', 'funny0animals'),
             ('acctdesc', 'Funny animals video 🥰🥰🥰'),
             ('location', ''),
             ('following', '1'),
             ('followers', '15'),
             ('totaltweets', '276'),
             ('usercreatedts', '2022-09-24 17:40:19'),
             ('tweetid', '1578898032850075649'),
             ('tweetcreatedts', '2022-10-09 00:00:00'),
             ('retweetcount', '0'),
             ('text',
              'Funny cat #CuteCats #CatsOfTwitter #cutevideo #Lyman #sexymonday #Kherson #1004DayWithJeonghan #Crypto #Kyiv #Zelensky #wealth #Bitcoin #지구에서년지성아환영해 #CrimeanBridge #кримськийміст https://t.co/waXCl5WeBE'),
             ('hashtags',
              "[{'text': 'CuteCats', 'indices': [10, 19]}, {'text': 'CatsOfTwitter', 'indices': [20, 34]}, {'text': 'cutevideo', 'indices': [35, 45]}, {'text': 'Lyman', 'indices': [46, 52]}, {'text': 'sexymonday', 'indices': 

Tokenize tweet by removing puntuation and stop words in order to have only meaningful words to understand the meaning of the tweet. I also put all the words lower case and remove the link at the end of the tweets.

In [9]:
!pip install clean-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 2.0 MB/s 
     |████████████████████████████████| 53 kB 1.5 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=d06db0a4b9b0c809a651292bf23018074eb4adc47db76725a0a5fd1801d9d3dc
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [10]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from string import punctuation
from cleantext import clean

nltk.download('stopwords')
stopw = set(stopwords.words('english'))

tokenizer = TweetTokenizer()
#tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation and word not in stopw]
tokenize = lambda text: [word for word in tokenizer.tokenize(clean(text, no_urls=True, no_emails=True, no_emoji=True, no_line_breaks=True, replace_with_email="", replace_with_url="")) if word not in punctuation]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
import numpy as np

def sort(list):
  list.sort()
  return list

en_tweets = rdd.filter(lambda x: x.get("language")=='en').map(lambda x: (x, x.get("text")))

tweets_tokenized = en_tweets.map(lambda x: (x[0].get("text"), tokenize(x[1])))

tweets_tokenized.first()

('Funny cat #CuteCats #CatsOfTwitter #cutevideo #Lyman #sexymonday #Kherson #1004DayWithJeonghan #Crypto #Kyiv #Zelensky #wealth #Bitcoin #지구에서년지성아환영해 #CrimeanBridge #кримськийміст https://t.co/waXCl5WeBE',
 ['funny',
  'cat',
  '#cutecats',
  '#catsoftwitter',
  '#cutevideo',
  '#lyman',
  '#sexymonday',
  '#kherson',
  '#1004daywithjeonghan',
  '#crypto',
  '#kyiv',
  '#zelensky',
  '#wealth',
  '#bitcoin',
  '#crimeanbridge'])

In [14]:
from pyspark.sql.types import StructType,StructField, StringType, ArrayType

dfSchema = StructType([       
    StructField('tweet', StringType(), True),
    StructField('tokens', ArrayType(StringType()), True)
])

tmp = tweets_tokenized.map(lambda x: (x[0], x[1]))
tweetsDF = spark.createDataFrame(tmp, dfSchema)

tweetsDF.show()

+--------------------+--------------------+
|               tweet|              tokens|
+--------------------+--------------------+
|Funny cat #CuteCa...|[funny, cat, #cut...|
|#SyriaTalks #Refu...|[#syriatalks, #re...|
|What's your NATO ...|[what's, your, na...|
|Happy 70th birthd...|[happy, 70th, bir...|
|Zaporizhzhia. 9th...|[zaporizhzhia, 9t...|
|Vladimir #Putin’s...|[vladimir, #putin...|
|"The bridge to #C...|[the, bridge, to,...|
|They specially ta...|[they, specially,...|
|#russia lukin lik...|[#russia, lukin, ...|
|Happy 70th birthd...|[happy, 70th, bir...|
|@ZelenskyyUa @KGe...|[@zelenskyyua, @k...|
|As #CrimeaBridge ...|[as, #crimeabridg...|
|Oh, my god, I hat...|[oh, my, god, i, ...|
|It was a very tou...|[it, was, a, very...|
|Right now, 10 Rus...|[right, now, 10, ...|
|@AVindman I think...|[@avindman, i, th...|
|🗣️help your self...|[help, your, self...|
|Happy 70th birthd...|[happy, 70th, bir...|
|The shelling of r...|[the, shelling, o...|
|#odessa #odesa #u...|[#odessa, #

## *K*-Shingles

In [20]:
K = int(input("Enter the number K for shingles dimension: "))

def build_shingles(tokens, K):
  shingles = list()
  tmp = ""

  for i in range(len(tokens)):
    tmp = tokens[i]
    if((len(tokens) - (i)) < K):
      break;
    for k in range(K - 1):
      tmp += " " + tokens[i + (k + 1)]
    shingles.append(tmp)

  return shingles

shingles_rdd = tweetsDF.rdd.map(lambda x: (x[0], sort(build_shingles(x[1], K))))
shingles_rdd.first()

Enter the number K for shingles dimension: 3


('Funny cat #CuteCats #CatsOfTwitter #cutevideo #Lyman #sexymonday #Kherson #1004DayWithJeonghan #Crypto #Kyiv #Zelensky #wealth #Bitcoin #지구에서년지성아환영해 #CrimeanBridge #кримськийміст https://t.co/waXCl5WeBE',
 ['#1004daywithjeonghan #crypto #kyiv',
  '#catsoftwitter #cutevideo #lyman',
  '#crypto #kyiv #zelensky',
  '#cutecats #catsoftwitter #cutevideo',
  '#cutevideo #lyman #sexymonday',
  '#kherson #1004daywithjeonghan #crypto',
  '#kyiv #zelensky #wealth',
  '#lyman #sexymonday #kherson',
  '#sexymonday #kherson #1004daywithjeonghan',
  '#wealth #bitcoin #crimeanbridge',
  '#zelensky #wealth #bitcoin',
  'cat #cutecats #catsoftwitter',
  'funny cat #cutecats'])

In [36]:
shingles_bag = sort(list(set(shingles_rdd.flatMap(lambda x: x[1]).collect())))
shingles_bag[0:10]

['###europeanunion are likely',
 '##brazil #lulano1turno #crimeanbridge',
 '##fakenews #humor #humour',
 '##istandwithukraine #russiainvadedukraine #russiaiscommitingenocide',
 '##kerch #crimeanbridge #crimea',
 '##latvia #lithuania #poland',
 '##navigation #google #maps',
 '##rosenberg #hakim #deaf_government',
 '##russia either people',
 '##swiped had its']

I use integers from 0 to *n* (with n equal to the length of shgles_bag) to store the shingles list

In [37]:
shingles_bag_id = list(range(0, len(shingles_bag)))

492968

In [39]:
def convertShingles(shingles, BoS, BoS_id):
  listToReturn = list()
  start_index = 0;

  for s in shingles:
    for i in range(start_index, len(BoS)):
      if(s == BoS[i]):
        listToReturn.append(BoS_id[i])
        start_index = i
        break;

  return listToReturn

shingles_int_rdd = shingles_rdd.map(lambda x: (x[0], convertShingles(x[1], shingles_bag, shingles_bag_id)))
shingles_int_rdd.take(10)

[('Funny cat #CuteCats #CatsOfTwitter #cutevideo #Lyman #sexymonday #Kherson #1004DayWithJeonghan #Crypto #Kyiv #Zelensky #wealth #Bitcoin #지구에서년지성아환영해 #CrimeanBridge #кримськийміст https://t.co/waXCl5WeBE',
  [30,
   5944,
   10305,
   10539,
   10542,
   20985,
   22120,
   23476,
   43211,
   58590,
   60753,
   156302,
   212547]),
 ('#SyriaTalks #RefugeeCrisis #Putin  1 author encyclopedia site:#Ummah-state to #Dictatorship https://t.co/Bf7gJ1HZOG https://t.co/rexwGyzjab',
  [31372, 33934, 46309, 56179, 63296, 131122, 189009, 376860]),
 ("What's your NATO plan?\n\n#nato #ww3 #ukraine #war #russia #UkraineRussiaWar https://t.co/IYpLrmwMpX",
  [25998, 50397, 58090, 59412, 298117, 337812, 471067, 490936]),
 ('Happy 70th birthday, Mr Putin\n... here is the fireworks display from #Crimea, #Ukraine \n\nPS - your tanks are running out of diesel fuel🤣🤣🤣 https://t.co/CYqrYsdchh',
  [8212,
   52247,
   62454,
   70039,
   123012,
   141558,
   181170,
   200981,
   209361,
   222047,
   229